In [10]:
#Install Required Libraries
!pip install pandas numpy scikit-learn tensorflow xgboost prophet pyod plotly seaborn matplotlib shap -q

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print(" AI Libraries Installed Successfully!")

 AI Libraries Installed Successfully!


In [15]:
#  Data Processor
class WaterCrisisAIDataProcessor:
    """
    AI-focused data processor with advanced feature engineering
    """
    
    def __init__(self, filepath):
        self.filepath = filepath
        self.df = None
        self.df_features = None
        self.country_metadata = {}
        
    def load_and_engineer_features(self):
        """Load data and create AI-ready features"""
        print("Loading data for AI analysis...")
        
        # Load data
        self.df = pd.read_csv(self.filepath, skiprows=4)
        self.df.columns = [col.strip().replace('"', '') for col in self.df.columns]
        
        # Get year columns
        year_cols = [col for col in self.df.columns if col.isdigit()]
        self.years = [int(y) for y in year_cols]
        
        # Create AI features
        self._create_ai_features()
        print(f" Data loaded with {len(self.df_features)} AI features")
        
        return self.df_features
    
    def _create_ai_features(self):
        """Create comprehensive features for AI models"""
        features_list = []
        
        for idx, row in self.df.iterrows():
            country = row['Country Name']
            country_code = row['Country Code']
            
            # Extract time series data
            ts_data = []
            valid_years = []
            for year in self.years:
                value = row.get(str(year))
                if pd.notna(value):
                    ts_data.append(float(value))
                    valid_years.append(year)
            
            if len(ts_data) < 10:  # Skip countries with insufficient data
                continue
            
            # Create features dictionary
            features = {
                'country': country,
                'country_code': country_code,
                'timeseries': ts_data,
                'years': valid_years,
                'latest_value': ts_data[-1] if ts_data else np.nan,
                'first_value': ts_data[0] if ts_data else np.nan,
                'trend_slope': self._calculate_trend_slope(ts_data, valid_years),
                'volatility': np.std(ts_data) if len(ts_data) > 1 else 0,
                'max_value': np.max(ts_data),
                'min_value': np.min(ts_data),
                'mean_value': np.mean(ts_data),
                'median_value': np.median(ts_data),
                'pct_change_10yr': self._calculate_percent_change(ts_data, valid_years, 10),
                'pct_change_20yr': self._calculate_percent_change(ts_data, valid_years, 20),
                'crisis_risk_score': self._calculate_crisis_risk(ts_data, valid_years),
                'is_extreme_high': 1 if ts_data[-1] > 100 else 0,
                'is_high_stress': 1 if ts_data[-1] > 50 else 0,
                'is_low_stress': 1 if ts_data[-1] < 10 else 0,
                'acceleration': self._calculate_acceleration(ts_data, valid_years)
            }
            
            # Add statistical features
            features.update(self._calculate_statistical_features(ts_data))
            
            features_list.append(features)
            
            # Store metadata
            self.country_metadata[country] = features
        
        self.df_features = pd.DataFrame(features_list)
        return self.df_features
    
    def _calculate_trend_slope(self, values, years):
        """Calculate linear trend slope"""
        if len(values) < 2:
            return 0
        x = np.array(years)
        y = np.array(values)
        slope = np.polyfit(x, y, 1)[0]
        return slope
    
    def _calculate_percent_change(self, values, years, period_years):
        """Calculate percentage change over specified period"""
        if len(values) < 2:
            return 0
        
        recent_idx = -1
        past_idx = max(-len(values), -period_years - 1)
        
        if past_idx >= -len(values):
            past_value = values[past_idx]
            recent_value = values[recent_idx]
            if past_value != 0:
                return ((recent_value - past_value) / past_value) * 100
        return 0
    
    def _calculate_crisis_risk(self, values, years):
        """Calculate risk score (0-100) for water crisis"""
        if len(values) < 5:
            return 0
        
        risk_score = 0
        
        # Latest value weight (40%)
        latest = values[-1]
        if latest > 100:
            risk_score += 40
        elif latest > 75:
            risk_score += 30
        elif latest > 50:
            risk_score += 20
        elif latest > 25:
            risk_score += 10
        
        # Trend weight (30%)
        trend = self._calculate_trend_slope(values, years)
        if trend > 2:
            risk_score += 30
        elif trend > 1:
            risk_score += 20
        elif trend > 0.5:
            risk_score += 10
        
        # Volatility weight (15%)
        volatility = np.std(values[-5:]) if len(values) >= 5 else np.std(values)
        if volatility > 15:
            risk_score += 15
        elif volatility > 10:
            risk_score += 10
        elif volatility > 5:
            risk_score += 5
        
        # Acceleration weight (15%)
        acceleration = self._calculate_acceleration(values, years)
        if acceleration > 1:
            risk_score += 15
        elif acceleration > 0.5:
            risk_score += 10
        elif acceleration > 0:
            risk_score += 5
        
        return min(100, risk_score)
    
    def _calculate_acceleration(self, values, years):
        """Calculate acceleration of change"""
        if len(values) < 10:
            return 0
        
        recent_slope = self._calculate_trend_slope(values[-5:], years[-5:]) if len(values) >= 5 else 0
        earlier_slope = self._calculate_trend_slope(values[-10:-5], years[-10:-5]) if len(values) >= 10 else 0
        
        return recent_slope - earlier_slope
    
    def _calculate_statistical_features(self, values):
        """Calculate various statistical features"""
        if len(values) < 3:
            return {}
        
        return {
            'skewness': pd.Series(values).skew(),
            'kurtosis': pd.Series(values).kurtosis(),
            'q1': np.percentile(values, 25),
            'q3': np.percentile(values, 75),
            'iqr': np.percentile(values, 75) - np.percentile(values, 25),
            'cv': np.std(values) / np.mean(values) if np.mean(values) != 0 else 0
        }
    
    def get_country_features(self, country_name):
        """Get features for specific country"""
        return self.df_features[self.df_features['country'] == country_name]
    
    def get_risk_categories(self):
        """Categorize countries by risk level"""
        self.df_features['risk_category'] = pd.cut(
            self.df_features['crisis_risk_score'],
            bins=[0, 25, 50, 75, 100],
            labels=['Low', 'Medium', 'High', 'Critical']
        )
        return self.df_features['risk_category'].value_counts()

In [ ]:
# Water Crisis Predictor - Machine Learning Models
class WaterCrisisPredictor:
    """
    AI models for predicting water crises
    """
    
    def __init__(self, data_processor):
        self.processor = data_processor
        self.df_features = data_processor.df_features
        self.models = {}
        self.predictions = {}
        
        from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
        from sklearn.cluster import KMeans, DBSCAN
        from sklearn.preprocessing import StandardScaler
        from sklearn.decomposition import PCA
        from pyod.models.iforest import IForest
        from xgboost import XGBClassifier
        
        self.RandomForestClassifier = RandomForestClassifier
        self.GradientBoostingClassifier = GradientBoostingClassifier
        self.KMeans = KMeans
        self.DBSCAN = DBSCAN
        self.StandardScaler = StandardScaler
        self.PCA = PCA
        self.IForest = IForest
        self.XGBClassifier = XGBClassifier
    
    def predict_crisis_risk(self, forecast_years=5):
        """
        Predict which countries will face water crises in next N years
        """
        print(" Predicting water crisis risks...")
        
        # Prepare features for prediction
        X = self._prepare_prediction_features()
        
        # Create labels (crisis = 1 if risk_score > 70 or value > 100)
        y = (self.df_features['crisis_risk_score'] > 70).astype(int)
        
        # Train Random Forest
        rf_model = self.RandomForestClassifier(
            n_estimators=100,
            random_state=42,
            max_depth=10
        )
        rf_model.fit(X, y)
        
        # Train XGBoost
        xgb_model = self.XGBClassifier(
            n_estimators=100,
            random_state=42,
            max_depth=8,
            learning_rate=0.1
        )
        xgb_model.fit(X, y)
        
        # Store models
        self.models['random_forest'] = rf_model
        self.models['xgboost'] = xgb_model
        
        # Make predictions
        rf_predictions = rf_model.predict_proba(X)[:, 1]
        xgb_predictions = xgb_model.predict_proba(X)[:, 1]
        
        # Ensemble prediction
        ensemble_pred = (rf_predictions + xgb_predictions) / 2
        
        # Add predictions to dataframe
        self.df_features['crisis_probability'] = ensemble_pred
        self.df_features['predicted_crisis'] = (ensemble_pred > 0.7).astype(int)
        
        # Identify high-risk countries
        high_risk = self.df_features[
            (self.df_features['crisis_probability'] > 0.7) & 
            (self.df_features['latest_value'] > 50)
        ].sort_values('crisis_probability', ascending=False)
        
        # Forecast future values using trend projection
        self._forecast_future_values(forecast_years)
        
        print(f"Identified {len(high_risk)} high-risk countries")
        
        return high_risk
    
    def _prepare_prediction_features(self):
        """Prepare numerical features for ML models"""
        feature_cols = [
            'latest_value', 'trend_slope', 'volatility', 'max_value',
            'mean_value', 'pct_change_10yr', 'pct_change_20yr',
            'crisis_risk_score', 'acceleration', 'skewness', 'kurtosis',
            'iqr', 'cv'
        ]
        
        # Fill missing values
        X = self.df_features[feature_cols].fillna(0)
        
        # Scale features
        scaler = self.StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        return X_scaled
    
    def _forecast_future_values(self, years_ahead):
        """Forecast future water withdrawal values"""
        forecasts = []
        
        for idx, row in self.df_features.iterrows():
            trend = row['trend_slope']
            current = row['latest_value']
            acceleration = row['acceleration']
            
            # Simple linear forecast with acceleration
            forecast_vals = []
            for year in range(1, years_ahead + 1):
                # Quadratic projection: current + trend*year + 0.5*acceleration*year^2
                forecast = current + trend*year + 0.5*acceleration*(year**2)
                forecast_vals.append(max(0, forecast))  # Don't go below 0
            
            forecasts.append({
                'country': row['country'],
                'current_value': current,
                f'forecast_{years_ahead}yr': forecast_vals[-1] if forecast_vals else current,
                'forecast_trajectory': forecast_vals
            })
        
        self.forecasts_df = pd.DataFrame(forecasts)
        
        # Merge with main dataframe
        self.df_features = self.df_features.merge(
            self.forecasts_df[['country', f'forecast_{years_ahead}yr']],
            on='country',
            how='left'
        )
        
        # Identify countries where forecast exceeds 100%
        self.df_features['will_exceed_100'] = (
            self.df_features[f'forecast_{years_ahead}yr'] > 100
        ).astype(int)
    
    def cluster_countries(self, n_clusters=5):
        """
        Cluster countries by water usage patterns using K-means
        """
        print(f" Clustering countries into {n_clusters} groups...")
        
        # Prepare clustering features
        cluster_features = [
            'latest_value', 'trend_slope', 'volatility', 'mean_value',
            'pct_change_10yr', 'crisis_risk_score', 'skewness'
        ]
        
        X_cluster = self.df_features[cluster_features].fillna(0)
        
        # Scale features
        scaler = self.StandardScaler()
        X_scaled = scaler.fit_transform(X_cluster)
        
        # Apply PCA for visualization
        pca = self.PCA(n_components=2)
        X_pca = pca.fit_transform(X_scaled)
        
        # Apply K-means clustering
        kmeans = self.KMeans(n_clusters=n_clusters, random_state=42)
        clusters = kmeans.fit_predict(X_scaled)
        
        # Add clusters to dataframe
        self.df_features['cluster'] = clusters
        self.df_features['pca_x'] = X_pca[:, 0]
        self.df_features['pca_y'] = X_pca[:, 1]
        
        # Analyze cluster characteristics
        cluster_analysis = self._analyze_clusters(clusters)
        
        return clusters, cluster_analysis
    
    def _analyze_clusters(self, clusters):
        """Analyze characteristics of each cluster"""
        analysis = {}
        
        for cluster_id in np.unique(clusters):
            cluster_data = self.df_features[self.df_features['cluster'] == cluster_id]
            
            analysis[cluster_id] = {
                'size': len(cluster_data),
                'avg_withdrawal': cluster_data['latest_value'].mean(),
                'avg_trend': cluster_data['trend_slope'].mean(),
                'avg_risk': cluster_data['crisis_risk_score'].mean(),
                'countries': cluster_data['country'].tolist()[:10],  # First 10 countries
                'characteristics': self._describe_cluster(cluster_data)
            }
        
        return analysis
    
    def _describe_cluster(self, cluster_data):
        """Generate natural language description of cluster"""
        avg_withdrawal = cluster_data['latest_value'].mean()
        avg_trend = cluster_data['trend_slope'].mean()
        avg_risk = cluster_data['crisis_risk_score'].mean()
        
        if avg_withdrawal > 100:
            desc = "Extreme water stress countries"
        elif avg_withdrawal > 50:
            desc = "High water stress countries"
        elif avg_withdrawal > 25:
            desc = "Moderate water use countries"
        else:
            desc = "Low water use countries"
        
        if avg_trend > 1:
            desc += " with rapidly increasing withdrawals"
        elif avg_trend > 0:
            desc += " with slowly increasing withdrawals"
        elif avg_trend < -1:
            desc += " with rapidly decreasing withdrawals"
        else:
            desc += " with stable or decreasing withdrawals"
        
        if avg_risk > 75:
            desc += " (CRITICAL RISK)"
        elif avg_risk > 50:
            desc += " (HIGH RISK)"
        
        return desc
    
    def detect_anomalies(self, contamination=0.1):
        """
        Detect unusual changes in withdrawal patterns using Isolation Forest
        """
        print(" Detecting anomalous patterns...")
        
        # Features for anomaly detection
        anomaly_features = [
            'latest_value', 'trend_slope', 'volatility', 
            'pct_change_10yr', 'acceleration', 'skewness'
        ]
        
        X_anomaly = self.df_features[anomaly_features].fillna(0)
        
        # Scale features
        scaler = self.StandardScaler()
        X_scaled = scaler.fit_transform(X_anomaly)
        
        # Apply Isolation Forest
        iso_forest = self.IForest(contamination=contamination, random_state=42)
        anomalies = iso_forest.fit_predict(X_scaled)
        
        # -1 for anomalies, 1 for normal
        self.df_features['is_anomaly'] = (anomalies == -1).astype(int)
        
        # Get anomaly scores
        self.df_features['anomaly_score'] = iso_forest.decision_function(X_scaled)
        
        anomalous_countries = self.df_features[
            self.df_features['is_anomaly'] == 1
        ]['country'].tolist()
        
        print(f" Detected {len(anomalous_countries)} anomalous countries")
        
        return anomalous_countries
    
    def get_recommendations(self, country_name, n_recommendations=5):
        """
        Recommend water conservation strategies based on similar countries
        """
        if country_name not in self.df_features['country'].values:
            return f"Country {country_name} not found in data"
        
        # Get target country features
        target_country = self.df_features[
            self.df_features['country'] == country_name
        ].iloc[0]
        
        # Find similar countries (same cluster, similar risk level)
        cluster_id = target_country['cluster']
        similar_countries = self.df_features[
            (self.df_features['cluster'] == cluster_id) &
            (self.df_features['country'] != country_name)
        ]
        
        if len(similar_countries) == 0:
            # Find countries with similar withdrawal levels
            target_value = target_country['latest_value']
            similar_countries = self.df_features[
                (self.df_features['country'] != country_name) &
                (abs(self.df_features['latest_value'] - target_value) < 10)
            ]
        
        # Sort by similarity (closest in latest_value and trend)
        similar_countries['similarity_score'] = similar_countries.apply(
            lambda row: 1 / (1 + abs(row['latest_value'] - target_country['latest_value']) + 
                            abs(row['trend_slope'] - target_country['trend_slope'])),
            axis=1
        )
        
        similar_countries = similar_countries.sort_values(
            'similarity_score', ascending=False
        ).head(n_recommendations)
        
        # Generate recommendations
        recommendations = self._generate_recommendation_text(
            target_country, similar_countries
        )
        
        return recommendations
    
    def _generate_recommendation_text(self, target_country, similar_countries):
        """Generate natural language recommendations"""
        country_name = target_country['country']
        current_value = target_country['latest_value']
        risk_score = target_country['crisis_risk_score']
        
        recommendations = []
        
        # Base recommendation based on current status
        if current_value > 100:
            recommendations.append(f" **CRITICAL**: {country_name}'s water withdrawals exceed 100% of internal resources. Immediate action required.")
            recommendations.append(" **Recommendation**: Implement emergency water conservation measures, explore alternative water sources, and consider importing water.")
        elif current_value > 75:
            recommendations.append(f" **HIGH STRESS**: {country_name} is using over 75% of its internal water resources.")
            recommendations.append(" **Recommendation**: Strengthen water governance, invest in water-saving technologies, and promote water recycling.")
        elif current_value > 50:
            recommendations.append(f" **MEDIUM-HIGH**: {country_name} uses over 50% of internal water resources.")
            recommendations.append("**Recommendation**: Improve irrigation efficiency, fix water distribution leaks, and implement water pricing reforms.")
        else:
            recommendations.append(f" **MANAGEABLE**: {country_name}'s water use is currently sustainable.")
            recommendations.append(" **Recommendation**: Continue monitoring and invest in long-term water infrastructure.")
        
        # Add learning from similar countries
        recommendations.append("\n**Learn from similar countries**:")
        
        for idx, row in similar_countries.iterrows():
            similar_country = row['country']
            similar_value = row['latest_value']
            similar_trend = row['trend_slope']
            
            if similar_trend < 0:  # Decreasing trend
                rec = f"  • {similar_country} reduced withdrawals from {similar_value + abs(similar_trend)*5:.1f}% to {similar_value:.1f}%"
                rec += " through efficient water management policies."
            else:
                rec = f"  • {similar_country} manages {similar_value:.1f}% withdrawals"
                rec += " with comprehensive water governance framework."
            
            recommendations.append(rec)
        
        # Specific strategies based on cluster
        cluster_id = target_country['cluster']
        cluster_strategies = {
            0: "Focus on agricultural water efficiency (drip irrigation)",
            1: "Industrial water recycling and reuse programs",
            2: "Urban water demand management and leak reduction",
            3: "Integrated water resource management across sectors",
            4: "Climate-resilient water infrastructure investment"
        }
        
        if cluster_id in cluster_strategies:
            recommendations.append(f"\n **Cluster-specific strategy**: {cluster_strategies[cluster_id]}")
        
        return "\n".join(recommendations)

In [9]:
# Visualization Dashboard
class WaterCrisisVisualizer:
    """
    Advanced visualizations for AI insights
    """
    
    def __init__(self, predictor):
        self.predictor = predictor
        self.df_features = predictor.df_features
        
    def plot_risk_map(self):
        """Create global risk map"""
        import plotly.express as px
        import plotly.graph_objects as go
        
        # Create risk categories for coloring
        risk_bins = [0, 25, 50, 75, 100]
        risk_labels = ['Low (0-25)', 'Medium (26-50)', 'High (51-75)', 'Critical (76-100)']
        self.df_features['risk_category'] = pd.cut(
            self.df_features['crisis_risk_score'],
            bins=risk_bins,
            labels=risk_labels
        )
        
        fig = px.choropleth(
            self.df_features,
            locations="country",
            locationmode="country names",
            color="risk_category",
            hover_name="country",
            hover_data=["latest_value", "crisis_risk_score", "crisis_probability"],
            color_discrete_map={
                'Low (0-25)': 'green',
                'Medium (26-50)': 'yellow',
                'High (51-75)': 'orange',
                'Critical (76-100)': 'red'
            },
            title=" Global Water Crisis Risk Map",
            labels={'risk_category': 'Risk Level'}
        )
        
        fig.update_layout(height=600)
        return fig
    
    def plot_cluster_analysis(self):
        """Visualize country clusters"""
        import plotly.express as px
        
        fig = px.scatter(
            self.df_features,
            x='pca_x',
            y='pca_y',
            color='cluster',
            hover_name='country',
            hover_data=['latest_value', 'trend_slope', 'crisis_risk_score'],
            title=' Country Clusters by Water Usage Patterns',
            labels={
                'pca_x': 'PCA Component 1',
                'pca_y': 'PCA Component 2',
                'cluster': 'Cluster',
                'latest_value': 'Current Withdrawal (%)'
            }
        )
        
        fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')))
        fig.update_layout(height=500)
        return fig
    
    def plot_crisis_predictions(self):
        """Visualize crisis predictions"""
        import plotly.graph_objects as go
        
        # Sort by crisis probability
        df_sorted = self.df_features.sort_values('crisis_probability', ascending=False).head(20)
        
        fig = go.Figure()
        
        fig.add_trace(go.Bar(
            x=df_sorted['crisis_probability'],
            y=df_sorted['country'],
            orientation='h',
            marker_color=df_sorted['crisis_probability'],
            text=df_sorted['crisis_probability'].round(3),
            textposition='auto',
            hovertemplate="<b>%{y}</b><br>Crisis Probability: %{x:.3f}<br>Current Withdrawal: %{customdata[0]:.1f}%<extra></extra>",
            customdata=df_sorted[['latest_value']]
        ))
        
        fig.update_layout(
            title=' Top 20 Countries Predicted for Water Crisis',
            xaxis_title='Crisis Probability (0-1)',
            yaxis_title='Country',
            height=600,
            yaxis={'categoryorder': 'total ascending'},
            coloraxis=dict(colorscale='RdYlGn_r')
        )
        
        # Add threshold line
        fig.add_vline(x=0.7, line_dash="dash", line_color="red", 
                     annotation_text="High Risk Threshold", 
                     annotation_position="top right")
        
        return fig
    
    def plot_anomaly_detection(self):
        """Visualize detected anomalies"""
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots
        
        anomalous = self.df_features[self.df_features['is_anomaly'] == 1]
        normal = self.df_features[self.df_features['is_anomaly'] == 0]
        
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=('Anomalous Countries', 'Normal Countries'),
            column_widths=[0.5, 0.5]
        )
        
        # Anomalous countries
        fig.add_trace(
            go.Scatter(
                x=anomalous['latest_value'],
                y=anomalous['trend_slope'],
                mode='markers',
                name='Anomalous',
                marker=dict(size=10, color='red', symbol='x'),
                text=anomalous['country'],
                hovertemplate="<b>%{text}</b><br>Withdrawal: %{x:.1f}%<br>Trend: %{y:.2f}<extra></extra>"
            ),
            row=1, col=1
        )
        
        # Normal countries
        fig.add_trace(
            go.Scatter(
                x=normal['latest_value'],
                y=normal['trend_slope'],
                mode='markers',
                name='Normal',
                marker=dict(size=8, color='blue', opacity=0.6),
                text=normal['country'],
                hovertemplate="<b>%{text}</b><br>Withdrawal: %{x:.1f}%<br>Trend: %{y:.2f}<extra></extra>"
            ),
            row=1, col=2
        )
        
        fig.update_xaxes(title_text="Current Withdrawal (%)", row=1, col=1)
        fig.update_xaxes(title_text="Current Withdrawal (%)", row=1, col=2)
        fig.update_yaxes(title_text="Trend Slope", row=1, col=1)
        fig.update_yaxes(title_text="Trend Slope", row=1, col=2)
        
        fig.update_layout(
            title=' Anomaly Detection in Water Withdrawal Patterns',
            height=500,
            showlegend=True
        )
        
        return fig
    
    def plot_trajectory_forecast(self, country_name, years_ahead=10):
        """Plot forecast trajectory for a country"""
        import plotly.graph_objects as go
        
        country_data = self.predictor.processor.country_metadata.get(country_name)
        if not country_data:
            return None
        
        years = country_data['years']
        values = country_data['timeseries']
        
        # Create forecast
        trend = country_data['trend_slope']
        current = values[-1]
        acceleration = country_data.get('acceleration', 0)
        
        forecast_years = list(range(years[-1] + 1, years[-1] + years_ahead + 1))
        forecast_values = []
        
        for i, year in enumerate(forecast_years, 1):
            forecast = current + trend*i + 0.5*acceleration*(i**2)
            forecast_values.append(max(0, forecast))
        
        # Combine historical and forecast
        all_years = years + forecast_years
        all_values = values + forecast_values
        
        fig = go.Figure()
        
        # Historical data
        fig.add_trace(go.Scatter(
            x=years,
            y=values,
            mode='lines+markers',
            name='Historical',
            line=dict(color='blue', width=3),
            marker=dict(size=8)
        ))
        
        # Forecast
        fig.add_trace(go.Scatter(
            x=forecast_years,
            y=forecast_values,
            mode='lines+markers',
            name='Forecast',
            line=dict(color='red', width=3, dash='dash'),
            marker=dict(size=8, symbol='diamond')
        ))
        
        # Add threshold lines
        fig.add_hline(y=100, line_dash="dot", line_color="red", 
                     annotation_text="Critical Threshold (100%)", 
                     annotation_position="bottom right")
        fig.add_hline(y=50, line_dash="dot", line_color="orange", 
                     annotation_text="High Stress (50%)", 
                     annotation_position="top right")
        
        fig.update_layout(
            title=f' Water Withdrawal Forecast for {country_name}',
            xaxis_title='Year',
            yaxis_title='Withdrawal (% of Internal Resources)',
            height=500,
            hovermode='x unified'
        )
        
        return fig

In [ ]:
 System Initialization
print(" INITIALIZING AI-POWERED WATER CRISIS PREDICTION SYSTEM")
print("="*70)

# Initialize the system
filepath = "API_ER.H2O.FWTL.ZS_DS2_en_csv_v2_4288.csv"  # Update with your file path

# Step 1: Process data with AI features
processor = WaterCrisisAIDataProcessor(filepath)
df_features = processor.load_and_engineer_features()

print(f"\n Data Summary:")
print(f"• Countries analyzed: {len(df_features)}")
print(f"• Features created: {len(df_features.columns)}")
print(f"• Average risk score: {df_features['crisis_risk_score'].mean():.1f}")

# Step 2: Initialize AI predictor
predictor = WaterCrisisPredictor(processor)

# Step 3: Run all AI analyses
print("\n RUNNING AI ANALYSES...")
print("-"*50)

# 1. Predict crisis risks
high_risk_countries = predictor.predict_crisis_risk(forecast_years=5)
print(f"1️⃣ Crisis Prediction: {len(high_risk_countries)} high-risk countries identified")

# 2. Cluster countries
clusters, cluster_analysis = predictor.cluster_countries(n_clusters=5)
print(f"2️⃣ Country Clustering: 5 distinct patterns identified")

# 3. Detect anomalies
anomalous_countries = predictor.detect_anomalies(contamination=0.1)
print(f"3️⃣ Anomaly Detection: {len(anomalous_countries)} unusual patterns found")

# Step 4: Initialize visualizer
visualizer = WaterCrisisVisualizer(predictor)

print("\n AI SYSTEM INITIALIZED SUCCESSFULLY!")
print("="*70)

In [ ]:
# CELL 5: Interactive Analysis Interface
def interactive_ai_analysis():
    """Interactive interface for exploring AI insights"""
    
    print(" AI-POWERED WATER CRISIS ANALYSIS INTERFACE")
    print("="*60)
    print("\nAvailable Functions:")
    print("1.  View global risk map")
    print("2.  See crisis predictions")
    print("3.  Explore country clusters")
    print("4.  Check anomaly detection")
    print("5.  Get country-specific analysis")
    print("6.  Get recommendations for a country")
    print("7.  View forecast for a country")
    print("8.  Export all AI insights")
    print("\nType 'exit' to quit")
    print("="*60)
    
    while True:
        choice = input("\nEnter your choice (1-8): ").strip()
        
        if choice.lower() == 'exit':
            print(" Exiting AI analysis interface")
            break
        
        elif choice == '1':
            print(" Generating global risk map...")
            fig = visualizer.plot_risk_map()
            fig.show()
            
        elif choice == '2':
            print("  Displaying crisis predictions...")
            fig = visualizer.plot_crisis_predictions()
            fig.show()
            
            # Show top 5 high-risk countries
            print("\n Top 5 Highest Risk Countries:")
            top_5 = predictor.df_features.sort_values('crisis_probability', ascending=False).head(5)
            for idx, row in top_5.iterrows():
                print(f"  {row['country']}: {row['crisis_probability']:.3f} probability")
                
        elif choice == '3':
            print(" Displaying country clusters...")
            fig = visualizer.plot_cluster_analysis()
            fig.show()
            
            # Show cluster descriptions
            print("\n Cluster Descriptions:")
            for cluster_id, analysis in cluster_analysis.items():
                print(f"\nCluster {cluster_id}: {analysis['characteristics']}")
                print(f"  Countries: {', '.join(analysis['countries'][:5])}...")
                
        elif choice == '4':
            print("🔍 Displaying anomaly detection...")
            fig = visualizer.plot_anomaly_detection()
            fig.show()
            
            print(f"\n  Anomalous Countries ({len(anomalous_countries)}):")
            for country in anomalous_countries[:10]:  # Show first 10
                print(f"  • {country}")
            if len(anomalous_countries) > 10:
                print(f"  ... and {len(anomalous_countries) - 10} more")
                
        elif choice == '5':
            country = input("Enter country name: ").strip()
            if country in predictor.df_features['country'].values:
                country_data = predictor.df_features[predictor.df_features['country'] == country].iloc[0]
                
                print(f"\n AI ANALYSIS FOR: {country}")
                print("-"*40)
                print(f"Current Withdrawal: {country_data['latest_value']:.1f}%")
                print(f"Risk Score: {country_data['crisis_risk_score']:.1f}/100")
                print(f"Crisis Probability: {country_data.get('crisis_probability', 0):.3f}")
                print(f"Trend: {country_data['trend_slope']:.3f} (positive = increasing)")
                print(f"Cluster: {int(country_data['cluster'])}")
                print(f"Anomaly: {'YES' if country_data.get('is_anomaly', 0) == 1 else 'NO'}")
                
                # Get forecast
                if 'forecast_5yr' in country_data:
                    print(f"5-Year Forecast: {country_data['forecast_5yr']:.1f}%")
                    if country_data['forecast_5yr'] > 100:
                        print("  Forecast exceeds 100% - CRITICAL RISK")
                        
            else:
                print(f"Country '{country}' not found in data")
                
        elif choice == '6':
            country = input("Enter country name for recommendations: ").strip()
            recommendations = predictor.get_recommendations(country, n_recommendations=3)
            print(f"\n RECOMMENDATIONS FOR {country.upper()}:")
            print("-"*40)
            print(recommendations)
            
        elif choice == '7':
            country = input("Enter country name for forecast: ").strip()
            fig = visualizer.plot_trajectory_forecast(country, years_ahead=10)
            if fig:
                fig.show()
            else:
                print(f"Could not generate forecast for {country}")
                
        elif choice == '8':
            print(" Exporting AI insights...")
            export_ai_insights(predictor)
            
        else:
            print("Invalid choice. Please enter 1-8 or 'exit'")

def export_ai_insights(predictor):
    """Export all AI insights to files"""
    import json
    
    timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. Export predictions
    predictions_df = predictor.df_features[[
        'country', 'latest_value', 'crisis_risk_score', 
        'crisis_probability', 'predicted_crisis', 'cluster',
        'is_anomaly', 'forecast_5yr'
    ]]
    predictions_df.to_csv(f'water_crisis_predictions_{timestamp}.csv', index=False)
    
    # 2. Export high-risk countries
    high_risk = predictor.df_features[
        predictor.df_features['crisis_probability'] > 0.7
    ][['country', 'crisis_probability', 'latest_value', 'trend_slope']]
    high_risk.to_csv(f'high_risk_countries_{timestamp}.csv', index=False)
    
    # 3. Export cluster analysis
    cluster_summary = []
    for cluster_id in range(5):
        cluster_data = predictor.df_features[predictor.df_features['cluster'] == cluster_id]
        cluster_summary.append({
            'cluster': cluster_id,
            'count': len(cluster_data),
            'avg_withdrawal': cluster_data['latest_value'].mean(),
            'avg_risk': cluster_data['crisis_risk_score'].mean(),
            'representative_countries': cluster_data['country'].head(5).tolist()
        })
    
    with open(f'cluster_analysis_{timestamp}.json', 'w') as f:
        json.dump(cluster_summary, f, indent=2)
    
    # 4. Export anomalies
    anomalies = predictor.df_features[
        predictor.df_features['is_anomaly'] == 1
    ][['country', 'latest_value', 'trend_slope', 'anomaly_score']]
    anomalies.to_csv(f'anomalous_countries_{timestamp}.csv', index=False)
    
    print(f" AI insights exported with timestamp: {timestamp}")
    print(f" Files created:")
    print(f"  • water_crisis_predictions_{timestamp}.csv")
    print(f"  • high_risk_countries_{timestamp}.csv")
    print(f"  • cluster_analysis_{timestamp}.json")
    print(f"  • anomalous_countries_{timestamp}.csv")

# Run the interactive interface
interactive_ai_analysis()

In [ ]:
# CELL 6: Example AI Insights Summary
print(" AI INSIGHTS SUMMARY")
print("="*70)

# Summary statistics
print("\n1️⃣ CRISIS PREDICTION SUMMARY:")
print(f"   • Total countries analyzed: {len(predictor.df_features)}")
print(f"   • High-risk countries (prob > 0.7): {len(predictor.df_features[predictor.df_features['crisis_probability'] > 0.7])}")
print(f"   • Critical countries (withdrawal > 100%): {len(predictor.df_features[predictor.df_features['latest_value'] > 100])}")

print("\n2️⃣ CLUSTERING SUMMARY:")
for cluster_id in range(5):
    cluster_size = len(predictor.df_features[predictor.df_features['cluster'] == cluster_id])
    cluster_avg = predictor.df_features[predictor.df_features['cluster'] == cluster_id]['latest_value'].mean()
    print(f"   • Cluster {cluster_id}: {cluster_size} countries, avg withdrawal: {cluster_avg:.1f}%")

print("\n3️⃣ ANOMALY DETECTION:")
print(f"   • Anomalous countries detected: {len(anomalous_countries)}")
print(f"   • Example anomalies: {', '.join(anomalous_countries[:3])}")

print("\n4️⃣ TOP 5 HIGHEST RISK COUNTRIES:")
top_risks = predictor.df_features.sort_values('crisis_probability', ascending=False).head(5)
for idx, row in top_risks.iterrows():
    print(f"   • {row['country']}: Risk={row['crisis_risk_score']:.1f}, Prob={row['crisis_probability']:.3f}")

print("\n5️⃣ RECOMMENDATION EXAMPLES:")
sample_countries = ['India', 'China', 'United States', 'Saudi Arabia']
for country in sample_countries:
    if country in predictor.df_features['country'].values:
        risk = predictor.df_features[predictor.df_features['country'] == country]['crisis_risk_score'].values[0]
        prob = predictor.df_features[predictor.df_features['country'] == country]['crisis_probability'].values[0]
        print(f"   • {country}: Risk={risk:.1f}, Crisis Probability={prob:.3f}")

In [ ]:
# CELL 7: Model Evaluation and Explainability
class AI_Model_Evaluator:
    """
    Evaluate and explain AI model predictions
    """
    
    def __init__(self, predictor):
        self.predictor = predictor
        self.df_features = predictor.df_features
        
    def evaluate_model_performance(self):
        """Evaluate the AI model's performance"""
        from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        # Prepare data
        X = self.predictor._prepare_prediction_features()
        y_true = (self.df_features['crisis_risk_score'] > 70).astype(int)
        
        # Get predictions from Random Forest
        rf_model = self.predictor.models.get('random_forest')
        if rf_model:
            y_pred = rf_model.predict(X)
            y_pred_proba = rf_model.predict_proba(X)[:, 1]
            
            print(" RANDOM FOREST MODEL EVALUATION:")
            print("-"*40)
            
            # Classification report
            print("\n Classification Report:")
            print(classification_report(y_true, y_pred, target_names=['Low Risk', 'High Risk']))
            
            # ROC-AUC Score
            roc_auc = roc_auc_score(y_true, y_pred_proba)
            print(f" ROC-AUC Score: {roc_auc:.3f}")
            
            # Feature importance
            self._plot_feature_importance(rf_model)
            
            return roc_auc
    
    def _plot_feature_importance(self, model):
        """Plot feature importance from Random Forest"""
        feature_names = [
            'latest_value', 'trend_slope', 'volatility', 'max_value',
            'mean_value', 'pct_change_10yr', 'pct_change_20yr',
            'crisis_risk_score', 'acceleration', 'skewness', 'kurtosis',
            'iqr', 'cv'
        ]
        
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1]
        
        plt.figure(figsize=(10, 6))
        plt.title("Feature Importance in Crisis Prediction")
        plt.bar(range(len(indices)), importances[indices])
        plt.xticks(range(len(indices)), [feature_names[i] for i in indices], rotation=45, ha='right')
        plt.ylabel("Importance")
        plt.tight_layout()
        plt.show()
        
        # Print top 5 features
        print("\n Top 5 Most Important Features:")
        for i in range(5):
            print(f"  {i+1}. {feature_names[indices[i]]}: {importances[indices[i]]:.3f}")
    
    def explain_prediction(self, country_name):
        """Explain why a specific country is predicted as high-risk"""
        import shap
        
        if country_name not in self.df_features['country'].values:
            return f"Country {country_name} not found"
        
        # Get the model
        rf_model = self.predictor.models.get('random_forest')
        if not rf_model:
            return "Model not trained yet"
        
        # Prepare data
        X = self.predictor._prepare_prediction_features()
        country_idx = self.df_features[self.df_features['country'] == country_name].index[0]
        
        # Create SHAP explainer
        explainer = shap.TreeExplainer(rf_model)
        shap_values = explainer.shap_values(X)
        
        # Get prediction for this country
        prediction = rf_model.predict_proba(X[country_idx:country_idx+1])[0, 1]
        
        print(f" PREDICTION EXPLANATION FOR: {country_name}")
        print(f"   Crisis Probability: {prediction:.3f}")
        print("\n Feature Contributions:")
        
        # Feature contributions
        feature_names = [
            'Current Withdrawal', 'Trend Slope', 'Volatility', 'Max Value',
            'Mean Value', '10Y Change %', '20Y Change %',
            'Risk Score', 'Acceleration', 'Skewness', 'Kurtosis',
            'IQR', 'CV'
        ]
        
        # Get SHAP values for this instance
        shap_val = shap_values[1][country_idx]  # Class 1 (crisis)
        
        # Sort by absolute contribution
        contributions = sorted(zip(feature_names, shap_val), key=lambda x: abs(x[1]), reverse=True)
        
        for feature, contribution in contributions[:5]:  # Top 5 contributors
            direction = "increases" if contribution > 0 else "decreases"
            print(f"   • {feature}: {contribution:.3f} ({direction} crisis risk)")
        
        return prediction

# Initialize and run evaluation
print(" MODEL EVALUATION AND EXPLAINABILITY")
print("="*70)

evaluator = AI_Model_Evaluator(predictor)
roc_auc = evaluator.evaluate_model_performance()

# Explain predictions for sample countries
print("\n EXPLAINING PREDICTIONS:")
sample_countries = ['Saudi Arabia', 'United Arab Emirates', 'Qatar']
for country in sample_countries:
    if country in predictor.df_features['country'].values:
        print(f"\n{'-'*40}")
        evaluator.explain_prediction(country)